In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def advect_1d(u0, a, dx, dt, n_steps):
    """
    Advance solution u0 by n_steps of dt using upwind FV.
    
    u0      : initial cell-averages, shape (N,)
    a       : advection speed (scalar)
    dx      : cell width
    dt      : time step
    n_steps : number of time steps
    """
    N = len(u0)
    u = u0.copy()
    
    for _ in range(n_steps):
        # periodic extension
        u_ext = np.empty(N+2)
        u_ext[1:-1] = u
        u_ext[0]  = u[-1]
        u_ext[-1] = u[0]
        
        # compute fluxes at interfaces
        if a > 0:
            flux = a * u_ext[:-1]   # F_{i+1/2} = a u_i
        else:
            flux = a * u_ext[1:]    # F_{i+1/2} = a u_{i+1}
        
        # update: u_i^{n+1} = u_i^n - (dt/dx)*(F_{i+1/2} - F_{i-1/2})
        u -= (dt/dx) * (flux[1:] - flux[:-1])
    
    return u

# ——— set parameters ———
x_min, x_max = 0.0, 1.0
N      = 200
a      = 1.0               # advection speed
CFL    = 0.8
dx     = (x_max - x_min)/N
dt     = CFL * dx/abs(a)
T_final = 1.0
n_steps = int(np.ceil(T_final/dt))
dt     = T_final / n_steps  # adjust dt so that n_steps*dt = T_final

# ——— initial condition ———
x_centers = x_min + (np.arange(N)+0.5)*dx
u0 = np.exp(-200*(x_centers-0.5)**2)  # Gaussian hump

# ——— run solver ———
u_final = advect_1d(u0, a, dx, dt, n_steps)

# ——— plot ———
plt.figure(figsize=(6,3))
plt.plot(x_centers, u0,    label='t=0')
plt.plot(x_centers, u_final,label=f't={T_final:.2f}')
plt.legend()
plt.xlabel('x')
plt.ylabel('u')
plt.title('1D Linear Advection (upwind FV)')
plt.tight_layout()
plt.show()
